In [3]:
import json
import urllib.parse
import requests
import numpy as np
import pandas as pd


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
full_df = pd.DataFrame()

#set the column types for the data frames we are going to load
column_types={'complaint_type': np.unicode,
   'incident_zip': np.unicode,
  }

offset = 0
chunk_size = 100000
while True:
    #retrieve data in chunks using json api as it lets us filter out years
    #and select only certain columms
    url_base = 'https://data.cityofnewyork.us/resource/fhrw-4uyv.json?'
    url_base += '$limit=%s&$offset=%s' % (chunk_size,offset)
    select_clause = '$select=:id,created_date,complaint_type,incident_zip,borough'
    #create a filter to get only 2017 data
    date_clause = "created_date between '2017-01-01T00:00:00' and '2017-12-31T23:59:59'"
    where_clause = '$where=' + urllib.parse.quote(date_clause)
    #create the full url
    url = '%s&%s&%s' % (url_base,select_clause,where_clause)

    #now get the filtered data
    result = requests.get(url)

    #load the data in to a data frame and append it to the full one
    current_df = pd.read_json(result.text, dtype=column_types)
    data_read = current_df.shape[0]

    #remove rows with unspecified boroughs
    current_df = current_df[current_df.borough != 'Unspecified']
    #now add to the full one
    full_df = pd.concat([full_df,current_df])

    #now increase the offset and exit if we read less than the full amount
    offset += chunk_size
    if data_read < chunk_size:
        break




In [11]:
#Problem 1
#Consider only the 10 most common overall complaint types. For each borough,
#how many of each of those 10 types were there in 2017?

#get the top 10 complaints by type in a list
complaint_counts = full_df.groupby(['complaint_type'])[':id'].agg('count')
top_complaints = complaint_counts.nlargest(10).to_dict().keys()

#now use the top 10 complaints to filter the main list
filtered_top_complaint_df = full_df[full_df['complaint_type'].isin(top_complaints)]
#now get the top complaints per borough
borough_top_complaint_df = filtered_top_complaint_df.groupby(
                            ['borough','complaint_type'])[':id'].agg('count')
borough_top_complaint_df



borough        complaint_type         
BRONX          Blocked Driveway           23832
               HEAT/HOT WATER             73681
               Illegal Parking            15662
               Noise                       3132
               Noise - Residential        55557
               Noise - Street/Sidewalk    14053
               Street Condition           11976
               Street Light Condition     20542
               UNSANITARY CONDITION       24921
               Water System               10411
BROOKLYN       Blocked Driveway           47890
               HEAT/HOT WATER             72731
               Illegal Parking            53767
               Noise                      15541
               Noise - Residential        66037
               Noise - Street/Sidewalk    21257
               Street Condition           24948
               Street Light Condition     24600
               UNSANITARY CONDITION       27554
               Water System               17700
M

In [14]:
#Problem 2
#For the 10 most populous zip codes, how many of each of those 10 types were there in 2017?

#load a data frame with zipcode population information from the census
zipcode_column_types={'Zip Code ZCTA': np.unicode }
zipcode_population_df = pd.read_csv('2010_Census_Population_By_Zipcode_ZCTA.csv',
                                    dtype=zipcode_column_types) \
                                    .rename(columns={'Zip Code ZCTA': 'incident_zip',
                                            '2010 Census Population':'Zip Population'})

#get distinct nyc zipcodes (might want to replace the with a separate imported dataset)
zipcodes = full_df.incident_zip.unique()

#filter the population data set by the distinct zipcodes to just get nyc populations
nyc_zipcode_population_df = zipcode_population_df[zipcode_population_df['incident_zip'] \
                                                  .isin(zipcodes)]

#now get the top ten most populous zipcodes from nyc (this might need to be improved
#to be done outside our complaint set)
ten_largest_zipcodes_df = nyc_zipcode_population_df.nlargest(10, 'Zip Population')

#now merge the top ten zipcodes set with the top 10 complaint set we previously calculated for problem 1
complaints_in_top_zipcodes_df = ten_largest_zipcodes_df.merge(
                   filtered_top_complaint_df[['complaint_type','incident_zip']],
                   on='incident_zip')
#now get the aggegate count of complaint type for each of the 10 zipcodes
complaints_in_top_zipcodes_agg_df = complaints_in_top_zipcodes_df.groupby(
    ['incident_zip', 'complaint_type']).count()
complaints_in_top_zipcodes_agg_df



Zip Population
incident_zip complaint_type                         
10025        Blocked Driveway                    123
             HEAT/HOT WATER                     2276
             Illegal Parking                     716
             Noise                              1398
             Noise - Residential                2034
             Noise - Street/Sidewalk            1214
             Street Condition                    625
             Street Light Condition               19
             UNSANITARY CONDITION                638
             Water System                        478
10467        Blocked Driveway                   1995
             HEAT/HOT WATER                     6637
             Illegal Parking                     941
             Noise                               281
             Noise - Residential                5638
             Noise - Street/Sidewalk             709
             Street Condition                    603
             Street Light Condition              466
             UNSANITARY CONDITION               2254
             Water System                        572
11207        Blocked Driveway                   2037
             HEAT/HOT WATER                     2637
             Illegal Parking                    1450
             Noise                               192
             Noise - Residential                3013
             Noise - Street/Sidewalk             540
             Street Condition                   1106
             Street Light Condition             1008
             UNSANITARY CONDITION               1634
             Water System                        772
...                                              ...
11368        Blocked Driveway                   4226
             HEAT/HOT WATER                     1738
             Illegal Parking                    1205
             Noise                               147
             Noise - Residential                2475
             Noise - Street/Sidewalk             677
             Street Condition                    549
             Street Light Condition              496
             UNSANITARY CONDITION                636
             Water System                        606
11373        Blocked Driveway                   2541
             HEAT/HOT WATER                     4246
             Illegal Parking                    1231
             Noise                               279
             Noise - Residential                1788
             Noise - Street/Sidewalk             302
             Street Condition                    690
             Street Light Condition              407
             UNSANITARY CONDITION                603
             Water System                        354
11385        Blocked Driveway                   2946
             HEAT/HOT WATER                     1828
             Illegal Parking                    3986
             Noise                               523
             Noise - Residential                2498
             Noise - Street/Sidewalk             587
             Street Condition                   1225
             Street Light Condition              837
             UNSANITARY CONDITION                704
             Water System                       1229

[100 rows x 1 columns]

In [13]:
#Problem 3
#Considering all complaint types. Which boroughs are the biggest "complainers" relative
#to the size of the population in 2017?
#Meaning, calculate a complaint-index that adjusts for population of the borough.

#get borough population information for 2010 from a previously downloaded file
borough_pop_df = pd.read_csv('New_York_City_Population_By_Neighborhood_Tabulation_Areas.csv')
borough_pop_df = borough_pop_df[borough_pop_df['Year'] == 2010]

#convert the boroughs to uppercase
#borough_pop_df['Borough'] = map(lambda x: x.upper(), borough_pop_df['Borough'])
borough_pop_df['Borough'] = borough_pop_df['Borough'].str.upper()
#now aggregate across all areas of each borough to get total population per borough
aggregate_borough_pop_df = borough_pop_df.groupby(['Borough'])['Population'] \
                           .sum().reset_index()

#count the complaints per borough from the full list of complaints in to a new dataframe
aggregate_borough_complaint_df = full_df.groupby(['borough'])[':id'].count() \
         .reset_index().rename(columns={'borough': 'Borough', ':id':'Complaints'})
#now merge the borough complaint set with the population set so we have both at once
full_borough_complaint_df = aggregate_borough_pop_df.merge(
        aggregate_borough_complaint_df[['Complaints','Borough']], on='Borough')
#get total population
population = full_borough_complaint_df['Population'].sum()
#get total complaints
total_complaints = full_borough_complaint_df['Complaints'].sum()
#calculate complaints per person in each borough
complaints_pp = full_borough_complaint_df['Complaints'] / full_borough_complaint_df['Population']
full_borough_complaint_df['Complaints_PP'] = complaints_pp
#calculate complaint percentage divided by population percentage
complaint_percentage = full_borough_complaint_df['Complaints'] / total_complaints
population_percentage = full_borough_complaint_df['Population'] / population
#set the complaint index
full_borough_complaint_df['Complaint_Index'] = complaint_percentage / population_percentage
full_borough_complaint_df

,Borough,Population,Complaints,Complaints_PP,Complaint_Index
0,BRONX,1385108,455163,0.328612,1.117188
1,BROOKLYN,2504700,764054,0.305048,1.037077
2,MANHATTAN,1585873,470353,0.296589,1.008320
3,QUEENS,2230722,589174,0.264118,0.897927
4,STATEN ISLAND,468730,125907,0.268613,0.913208
